<a href="https://colab.research.google.com/github/ArushiAgrawal/DeepFake/blob/master/First_Order_Model_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Demo for paper "First Order Motion Model for Image Animation"

This works for pre-recorded videos. Not real time. 

**Clone repository**
Clone the souce code repository for first-order-model by Aliaksandr Siarohin, Stéphane Lathuilière, Sergey Tulyakov, Elisa Ricci and Nicu Sebe.

In [0]:
!git clone https://github.com/AliaksandrSiarohin/first-order-model

Cloning into 'first-order-model'...
remote: Enumerating objects: 236, done.
remote: Total 236 (delta 0), reused 0 (delta 0), pack-reused 236
Receiving objects: 100% (236/236), 71.46 MiB | 31.07 MiB/s, done.
Resolving deltas: 100% (117/117), done.


In [0]:
cd first-order-model

/content/first-order-model


**Add folder https://drive.google.com/drive/folders/1kZ1gCnpfU0BnpdU47pLM_TQ6RypDDqgw?usp=sharing  to your google drive.**

**Mount your Google drive folder on Colab**

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

**Upload your video (driving video)**

I am uploading the video through my local machine, we can also use something from the google drive. 

In [0]:
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
driving_video = imageio.mimread('your_video_name.mp4',memtest=False)
#Resize video to 256x256
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

Saving whatsapp-video-2020-06-14-at-61255-pm-1_fx5AIRnt_u5nV.mp4 to whatsapp-video-2020-06-14-at-61255-pm-1_fx5AIRnt_u5nV.mp4
User uploaded file "whatsapp-video-2020-06-14-at-61255-pm-1_fx5AIRnt_u5nV.mp4" with length 1185307 bytes


**Upload image to used as mask (Source Image)**

In [0]:
source_image = imageio.imread('/content/drive/My Drive/download.jpg')
#Resize image to 256x256
source_image = resize(source_image, (256, 256))[..., :3]

**Load driving video and source image**

In [0]:
def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani
    
#HTML(display(source_image, driving_video).to_html5_video())

**Create a model and load checkpoints**

In [0]:
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path='/content/drive/My Drive/first-order-motion-model/vox-cpk.pth.tar')

**Perform image animation**

In [0]:
from demo import make_animation
from skimage import img_as_ubyte

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

#save resulting video
imageio.mimsave('../generated.mp4', [img_as_ubyte(frame) for frame in predictions])
#video can be downloaded from /content folder

100%|██████████| 494/494 [00:19<00:00, 25.58it/s]


In [0]:
HTML(display(image, driving_video, predictions).to_html5_video())

**In the cell above we use relative keypoint displacement to animate the objects. We can use absolute coordinates instead,  but in this way all the object proporions will be inherited from the driving video. For example Putin haircut will be extended to match Trump haircut.**

In [0]:
predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=False, adapt_movement_scale=True)
HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 211/211 [00:07<00:00, 28.72it/s]
